In [2]:
import numpy as np 
from tqdm import tqdm # implementation of progress bars in jupyter notebooks
import sys 
sys.path.append('../../modules/')
import leaf_model as lm 


In [ ]:
def sensitivity_search(delta_min=0.2, kappa_min=0.2, lambda_min=0.2, chi_=0.10, N=400, n=5):
    # set up mesh of parameters (tau, gamma), rho_deltas=D_PM/D_SM, rho_kappas=K_SM/K_PM, rho_lambdas=L_PM/L
    taus   = np.exp(np.linspace(np.log(0.01), np.log(100), N))
    gammas = np.exp(np.linspace(np.log(0.01), np.log(100), N))
    rho_deltas  = np.linspace(delta_min, 1, n)
    rho_kappas  = np.linspace(kappa_min, 1, n)
    rho_lambdas = np.linspace(lambda_min, 1-lambda_min, n)
    # initialize leaf model object 
    leaf = lm.Leaf(1, 1, chi_, rho=(1,1,0.5), resolution=100)
    sensitivities  = np.zeros((N,N)) # container for sensitivities \eta
    # loop over taus and gammas
    for i in tqdm(range(N)):
        for j in range(N):
            # update leaf model parameters
            leaf.tau   = taus[i]
            leaf.gamma = gammas[j]
            leaf.rho = (1,1,0.5) # homogeneous mesophyll parameters (no spongy/palisade mesophyll contrast)
            # calculate homogeneous mesophyll solution
            domain, mf_solution = leaf.calculate_steady_state_solution()
            #
            samples = n**3
            drawdown = np.zeros(samples) # containers for relative drawdown (Ca - Ci)/Ca values between homogeneous and heterogeneous mesophyll solutions
            index = 0
            # loop over rho_deltas, rho_kappas, rho_lambdas
            for k in range(n):
                for l in range(n):
                    for m in range(n):
                        rho = (rho_deltas[k], rho_kappas[l], rho_lambdas[m])
                        leaf.rho = rho
                        domain, solution = leaf.calculate_steady_state_solution()
                        drawdown[index] = (mf_solution[0] - solution[0]) / (1 - mf_solution[0]) # calculate relative drawdown
                        index += 1
            sensitivities[i,j] = np.sqrt(np.sum(drawdown**2)/samples)
    return taus, gammas, sensitivities.T # taus on x-axis (2nd index) and gammas on y-axis (1st index)


taus, gammas, sensitivities = sensitivity_search()
np.savetxt('../saved_data/sensitivities.txt', sensitivities, delimiter=';', header='delta_min=0.2, kappa_min=0.2, lambda_min=0.2, zeta_compensation=0.15, N=400, n=5')

